## Conversion to fif: LL16

This script takes the raw, original data file we recieved from WashU and converts it to a fif file with the photodiode as a new stim channel


Just need to change fields in `prep_paths`

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [4]:
## prep paths

subject = 'LL16'
orig_data_fi = '/home/brooke/knight_server/remote/ecog/DATA_FOLDER/Loma_Linda/LL16/Datafiles/BCI/DAY2/Pacman Task/ECOG001/ECOGS001R01.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}/ieeg"

In [5]:
## load data

with b2k.BCI2kReader(orig_data_fi) as data: #opens a stream to the dat file
    my_signals = data.signals
    my_states = data.states
    my_params = data.parameters

In [7]:
## check structure if necessary

# my_states
my_params

{'SourceCh': 148,
 'SampleBlockSize': 20,
 'SamplingRate': 512,
 'ChannelNames': ['CH0',
  'CH1',
  'CH2',
  'CH3',
  'CH4',
  'CH5',
  'CH6',
  'CH7',
  'CH8',
  'CH9',
  'CH10',
  'CH11',
  'CH12',
  'CH13',
  'CH14',
  'CH15',
  'CH16',
  'CH17',
  'CH18',
  'CH19',
  'CH20',
  'CH21',
  'CH22',
  'CH23',
  'CH24',
  'CH25',
  'CH26',
  'CH27',
  'CH28',
  'CH29',
  'CH30',
  'CH31',
  'CH32',
  'CH33',
  'CH34',
  'CH35',
  'CH36',
  'CH37',
  'CH38',
  'CH39',
  'CH40',
  'CH41',
  'CH42',
  'CH43',
  'CH44',
  'CH45',
  'CH46',
  'CH47',
  'CH48',
  'CH49',
  'CH50',
  'CH51',
  'CH52',
  'CH53',
  'CH54',
  'CH55',
  'CH56',
  'CH57',
  'CH58',
  'CH59',
  'CH60',
  'CH61',
  'CH62',
  'CH63',
  'CH64',
  'CH65',
  'CH66',
  'CH67',
  'CH68',
  'CH69',
  'CH70',
  'CH71',
  'CH72',
  'CH73',
  'CH74',
  'CH75',
  'CH76',
  'CH77',
  'CH78',
  'CH79',
  'CH80',
  'CH81',
  'CH82',
  'CH83',
  'CH84',
  'CH85',
  'CH86',
  'CH87',
  'CH88',
  'CH89',
  'CH90',
  'CH91',
  'CH92',


In [8]:
## create info field 

info = mne.create_info(ch_names = my_params['ChannelNames'],
                       sfreq = my_params['SamplingRate'],
                       ch_types = 'seeg')

info

<Info | 7 non-empty values
 bads: []
 ch_names: CH0, CH1, CH2, CH3, CH4, CH5, CH6, CH7, CH8, CH9, CH10, CH11, ...
 chs: 148 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 148
 projs: []
 sfreq: 512.0 Hz
>

In [9]:
## create mne object
mne_df = mne.io.RawArray(my_signals, info)

Creating RawArray with float64 data, n_channels=148, n_times=940900
    Range : 0 ... 940899 =      0.000 ...  1837.693 secs
Ready.


In [10]:
## create stim channel and add it to mne object

info = mne.create_info(['STI'], my_params['SamplingRate'], ['stim'])
stim_raw = mne.io.RawArray( my_states['Trial_on_off'], info)
mne_df.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=940900
    Range : 0 ... 940899 =      0.000 ...  1837.693 secs
Ready.


<RawArray | 149 x 940900 (1837.7 s), ~1.04 GB, data loaded>

In [11]:
## save it

mne_df.save(f"{raw_data_dir}/{subject}_raw_ieeg.fif", overwrite = False)

Writing /home/brooke/pacman/raw_data/LL16/ieeg/LL16_raw_ieeg.fif
Closing /home/brooke/pacman/raw_data/LL16/ieeg/LL16_raw_ieeg.fif
[done]
